# Build a set of training data

Use a set of (most) common peptides to create inital data sets

- based on `Counter` over all outputs from search (here: MaxQuant)
   - keep based on threshold `FEAT_COMPLETNESS_CUTOFF` possible features
   - option: select samples based on `YEARS` (e.g. due constrain by a batch of strains)
   - collect in wide format data from output files

In [ ]:
from functools import partial
from pathlib import Path
import logging
import multiprocessing

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm_notebook

import vaep

import config

In [ ]:
from typing import List
def select_files_by_parent_folder(fpaths:List, years:List):
    selected = []
    for year_folder in years:
        # several passes, but not a bottle neck
        selected += [dump for dump in fpaths if year_folder in dump.parent.stem]
    return selected

## Setup

In [ ]:
RANDOM_SEED: int = 42  # Random seed for reproducibility
FEAT_COMPLETNESS_CUTOFF = 0.25 # Minimal proportion of samples which have to share a feature
YEARS = ['2017','2018', '2019', '2020']
SAMPLE_COL = 'Sample ID'
OUT_FOLDER = 'data/selected/'
FN_ID_OLD_NEW: str = 'data/rename/selected_old_new_id_mapping.csv' # selected samples with pride and original id

Select a specific config file

In [ ]:
# options = ['peptides', 'evidence', 'proteinGroups']
from config.training_data import peptides as cfg
# from config.training_data import evidence as cfg
# from config.training_data import proteinGroups as cfg

cfg_dict = {k: getattr(cfg, k) for k in dir(cfg) if not k.startswith('_')}
cfg_dict

Set defaults from file (allows to potentially overwrite parameters)

In [ ]:
# normal structure of config.py files
NAME = cfg.NAME
BASE_NAME = cfg.BASE_NAME

TYPES_DUMP = cfg.TYPES_DUMP
TYPES_COUNT = cfg.TYPES_COUNT

IDX_COLS_LONG = cfg.IDX_COLS_LONG

LOAD_DUMP = cfg.LOAD_DUMP

CounterClass = cfg.CounterClass
FNAME_COUNTER = cfg.FNAME_COUNTER

In [ ]:
out_folder = Path(OUT_FOLDER) / cfg.NAME
out_folder.mkdir(exist_ok=True, parents=True)

## Selected IDs

- currently only `Sample ID` is used
- path are to `.raw` raw files, not the output folder (could be changed)

In [ ]:
df_ids = pd.read_csv(FN_ID_OLD_NEW)
df_ids

## Counter

In [ ]:
counter = CounterClass(FNAME_COUNTER)
counts = counter.get_df_counts()
counts

In [ ]:
if TYPES_COUNT:
    counts = counts.convert_dtypes().astype({'Charge': int}) #
mask = counts['proportion'] >= FEAT_COMPLETNESS_CUTOFF
counts.loc[mask]

Based on selected samples, retain features that potentially could be in the subset

- if 1000 samples are selected, and given at treshold of 25%, one would need at least 250 observations

In [ ]:
treshold_counts = int(len(df_ids) * FEAT_COMPLETNESS_CUTOFF)
mask = counts['counts'] >= treshold_counts
counts.loc[mask]

In [ ]:
IDX_selected = counts.loc[mask].set_index('Sequence').index
IDX_selected

## Select Dumps

In [ ]:
selected_dumps = df_ids["Sample ID"]
selected_dumps = {k: counter.dumps[k] for k in selected_dumps}
selected_dumps = list(selected_dumps.items())
selected_dumps[:10]

potentially select meta data

In [ ]:
# selected_dumps = select_files_by_parent_folder(list(counter.dumps.values()), years=YEARS)
# print("Total number of files:", len(selected_dumps))
# selected_dumps[-10:]

## Collect in parallel

In [ ]:
# def load_fct(path):
#     s = (
#     pd.read_csv(path, index_col="Sequence", usecols=["Sequence", "Intensity"])
#     .notna()
#     .squeeze()
#     .astype(pd.Int8Dtype())
#     )
#     return s


def collect(folders, index, load_fct):
    current = multiprocessing.current_process()
    i = current._identity[0] % N_WORKERS + 1
    print(" ", end="", flush=True)

    failed = []
    all = pd.DataFrame(index=index)

    with tqdm_notebook(total=len(folders), position=i) as pbar:
        for id, path in folders:
            try:
                s = load_fct(path)
                s.name = id
                all = all.join(s, how='left')
            except FileNotFoundError:
                logging.warning(f"File not found: {path}")
                failed.append((id, path))
            except pd.errors.EmptyDataError:
                logging.warning(f"Empty file: {path}")
                failed.append((id, path))
            pbar.update(1)
            
    return all

## Collect intensities in parallel

In [ ]:
def load_fct(path):
    s = (
    pd.read_csv(path, index_col="Sequence", usecols=["Sequence", "Intensity"])
    .squeeze()
    .astype(pd.Int64Dtype())
    )
    return s

all = None # free memory

collect_intensities = partial(collect, index=IDX_selected, load_fct=load_fct)

N_WORKERS = 8

with multiprocessing.Pool(N_WORKERS) as p:
    all = list(
        tqdm_notebook(
            p.imap(collect_intensities,
                   np.array_split(selected_dumps, N_WORKERS)),
                   total=N_WORKERS,
        )
    )  
    
all = pd.concat(all, axis=1)
all

In [ ]:
all.memory_usage(deep=True).sum() / (2**20)

In [ ]:
%%time
fname = out_folder / config.insert_shape(all,  'intensities_wide_selected{}.pkl') 
all.to_pickle(fname)
fname

In [ ]:
%%time
all.to_csv(fname.with_suffix('.csv'), chunksize=1_000)

Samples as rows, feature columns as columns

- can fail due to memory

In [ ]:
# all = all.T # 

In [ ]:
# %%time
# fname = out_folder / config.insert_shape(all,  template='intensities_wide_selected{}.pkl', shape=(N, M)) 
# all.to_pickle(fname)